Data procession

In [6]:
zip_file_path = '/content/drive/MyDrive/datasets (1)/FakeAVCeleb_v1.2.zip'

In [9]:
!unzip /content/drive/MyDrive/datasets/generated_audio.zip

Streaming output truncated to the last 5000 lines.
  inflating: generated_audio/common_voices_prompts_from_conformer_fastspeech2_pwg_ljspeech/generated/gen_13087.wav  
  inflating: generated_audio/common_voices_prompts_from_conformer_fastspeech2_pwg_ljspeech/generated/gen_13508.wav  
  inflating: generated_audio/common_voices_prompts_from_conformer_fastspeech2_pwg_ljspeech/generated/gen_7532.wav  
  inflating: generated_audio/common_voices_prompts_from_conformer_fastspeech2_pwg_ljspeech/generated/gen_14267.wav  
  inflating: generated_audio/common_voices_prompts_from_conformer_fastspeech2_pwg_ljspeech/generated/gen_8601.wav  
  inflating: generated_audio/common_voices_prompts_from_conformer_fastspeech2_pwg_ljspeech/generated/gen_15179.wav  
  inflating: generated_audio/common_voices_prompts_from_conformer_fastspeech2_pwg_ljspeech/generated/gen_12616.wav  
  inflating: generated_audio/common_voices_prompts_from_conformer_fastspeech2_pwg_ljspeech/generated/gen_1143.wav  
  inflating: gen

In [16]:
!bzip2 -dk /content/drive/MyDrive/datasets/LJSpeech-1.1.tar.bz2

In [4]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ASVspoof-main')

In [5]:
from python_speech_features import mfcc
from CQCC.cqcc import cqcc
import scipy.io.wavfile as wav
import soundfile as sf
import os
import numpy as np
import pickle
import argparse
import librosa
import time
import gc
import argparse

from sklearn.mixture import GaussianMixture as GMM
from sklearn import preprocessing
from pdb import set_trace
from scipy import stats

In [23]:
def extract_cqcc(x, fs):
    x = x.reshape(x.shape[0], 1)  # for one-channel signal 
    B = 96
    fmax = fs/2
    fmin = fmax/2**9
    d = 16
    cf = 19
    ZsdD = 'ZsdD'
    CQcc, LogP_absCQT, TimeVec, FreqVec, Ures_LogP_absCQT, Ures_FreqVec, absCQT = cqcc(x, fs, B, fmax, fmin, d, cf, ZsdD)
    return CQcc, fmax, fmin

In [44]:
feats = []
file_dir = '/content/drive/MyDrive/wavs'

In [45]:
for file_path in os.listdir(file_dir):
  sig, rate = sf.read(os.path.join(file_dir, file_path))
  feat_cqcc, fmax, fmin = extract_cqcc(sig, rate)
  numframes = feat_cqcc.shape[0]
  winstep = 0.005
  winlen =  (len(sig) - winstep*rate*(numframes-1))/rate
  feat_mfcc = mfcc(sig,rate,winlen=winlen,winstep=winstep, lowfreq=fmin,highfreq=fmax)      # number of frames * number of cep
  feats.append((feat_cqcc, feat_mfcc, 'spoof'))

In [47]:
with open('/content/train_batch2.pkl', 'wb') as outfile:
    pickle.dump(feats, outfile)

In [48]:
!cp train_batch2.pkl /content/drive/MyDrive

In [46]:
len(feats)

1171

Training and testing GMMs

In [6]:
gmm_bon = GMM(n_components = 144, covariance_type='diag',n_init = 50) 
gmm_sp  = GMM(n_components = 144, covariance_type='diag',n_init = 50)

In [7]:
bondata = []
spdata = []

bon_test_data = []
sp_test_data = []

In [8]:
with open('/content/drive/MyDrive/train_batch1.pkl', 'rb') as infile:
  data = pickle.load(infile)
  counter = 0
  for feat_cqcc, feat_mfcc, label in data:
    # if (label == 'bonafide'):
    #   bondata.append(feats)
    # elif(label == 'spoof'):
    #   spdata.append(feats)
    if counter <= 2000:
      spdata.append(feat_mfcc)
      counter += 1
    else:
      sp_test_data.append(feat_mfcc)
      counter += 1
            
with open('/content/drive/MyDrive/train_batch2.pkl', 'rb') as infile:
  data_ = pickle.load(infile)
  counter = 0
  for feat_cqcc, feat_mfcc, label in data:
    # if (label == 'bonafide'):
    #   bondata.append(feats)
    # elif(label == 'spoof'):
    #   spdata.append(feats)
    if counter <= 1000:
      bondata.append(feat_mfcc)
      counter += 1
    else:
      bon_test_data.append(feat_mfcc)
      counter += 1

In [9]:
print(len(bondata))
print(len(spdata))

print(len(bon_test_data))
print(len(sp_test_data))

1001
2001
1633
633


In [ ]:
Xbon = np.vstack(bondata)
print('Bon feature stacked, shape as: ', Xbon.shape)
Xsp = np.vstack(spdata)
print('Sp feature stacked, shape as: ', Xsp.shape)

t0 = time.time()
gmm_bon.fit(Xbon)
print('Bon gmm trained, time spend:', time.time() - t0)

t0 = time.time()
gmm_sp.fit(Xsp)
print('Sp gmm trained, time spend:', time.time() - t0)

pickle.dump(gmm_bon, open('bon' + '.gmm', 'wb'))
pickle.dump(gmm_sp, open('sp' + '.gmm', 'wb'))
print('GMM model created')

!cp bon.gmm /content/drive/MyDrive
!cp sp.gmm /content/drive/MyDrive

Bon feature stacked, shape as:  (626777, 13)
Sp feature stacked, shape as:  (1242019, 13)


In [ ]:
predb = []
preds = []

In [ ]:
# with open('/content/drive/MyDrive/test.pkl', 'rb') as infile:
#   data_spoof = pickle.load(infile)
#   for feat_cqcc, feat_mfcc, label in data:
#     if (label == 'bonafide'):
#       bondata.append(feat_mfcc)
#     elif(label == 'spoof'):
#       spdata.append(feat_mfcc)

In [ ]:
j_bon = len(bon_test_data)
k_sp  = len(sp_test_data)

In [ ]:
for i in range(j_bon):
  if (i % 50 == 0):
    print('Evaluating Bon sample at',i/j_bon * 100, '%')
  X = bon_test_data[i]
  bscore = gmm_bon.score(X)
  sscore = gmm_sp.score(X)

  predb.append(bscore-sscore)

for i in range(k_sp):
  if (i % 50 == 0):
    print('Evaluating Sp sample at',i/k_sp * 100, '%')
  X = sp_test_data[i]
  bscore = gmm_bon.score(X)
  sscore = gmm_sp.score(X)

  preds.append(bscore-sscore)

predb1 = np.asarray(predb)
preds1 = np.asarray(preds)

predb1[predb1 < 0] = 0
predb1[predb1 > 0] = 1
predbresult1 = np.sum(predb1)
print(predbresult1, 'Bon samples were CORRECTLY evaluated out of', j_bon,'samples. Bon_Accuracy = ', predbresult1/j_bon )

preds1[preds1 > 0] = 0
preds1[preds1 < 0] = 1
predsresult = np.sum(preds1)
print(predsresult, 'Sp samples were CORRECTLY evaluated out of', k_sp, 'samples. Sp_Accuracy = ', predsresult/k_sp)# 0.4092

print('Total GMM Classifier Accuracy = ',(predbresult1 + predsresult)/(j_bon + k_sp))